<a href="https://colab.research.google.com/github/andonyns/air-quality/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Selección de dataset:

Descargar de [OpenAQ.org](https://openaq.org), utilizando APIs, los datos disponibles de Costa Rica en la última década (o para los años que haya datos), en todos los indicadores disponibles sobre la calidad del Aire.

También se deben descargar los datos de al menos otras cinco ciudades para las cuales deben incluir al menos 5 parámetros de calidad de aire.

Se debe brindar una explicación de cada parámetro analizado y buscar comparar ciudades que al menos compartan 3 parámetros para compararlos entre sí.


In [9]:
import requests



# 2. Tareas de limpieza y transformación:

Se deben realizar las tareas de limpieza y transformación necesarias para poder hacer un comparativo de la evolución de los diferentes indicadores de la calidad del aire en Costa Rica y las otras ciudades.



# 3. Implementación en Google Colab:

Realizar la implementación en Google Colab. Si existen problemas de desempeño, se puede optar por otro entorno, lo cual debe ser anotado en la documentación del notebook así como en la presentación.



# 4. Análisis y comparación:

Se debe realizar un análisis EDA que incluya análisis univariable y multivariable.

Analizar las tendencias de los indicadores para las diferentes ciudades y hacer comparaciones entre diferentes países y ciudades.

Incluir posibles correlaciones entre las variables y parámetros de calidad del aire de cada país/ciudad.

Utilizar diferentes tipos de visualizaciones relevantes para el análisis.



# 5. Conclusiones y Recomendaciones:

Extraer conclusiones sobre la evolución de la calidad del aire en Costa Rica y las ciudades seleccionadas, explicando cómo los datos sustentan estas conclusiones.

Buscar información sobre las políticas ambientales y regulaciones en estas ciudades y mostrar cómo los datos reflejan el efecto de estas políticas.